In [1]:
import pandas as pd
import random

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import numpy as np
import praw
import datetime
import json
import re
import emoji

[nltk_data] Downloading package punkt to /Users/aoife/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/aoife/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#  0 - Log into Reddit API

In [2]:
reddit = praw.Reddit(user_agent='VAD',
                     client_id='Que1ZJxBJmaWI7A4VvmZKw', client_secret="AhSItN-CRonRy8kjdeTf3PKUXtyZwg",
                     username='photosynthesising', password='Addie12345')

#  1 - Import Comments

In [3]:
# to start, the datasets need to be imported
# this is done by post, into a list of strings
# this makes it easy to continue the pre-process after

#  1.1 Dataset 1

In [43]:
# Post ID
post_id = "13jmu3f"
post = reddit.submission(id=post_id)

In [44]:
# Empty lists to store data
comments_list = []
karma_list = []
type_list = []

# Function to recursively collect comments, karma scores, and types
def get_comments(comment, comment_type):
    if isinstance(comment, praw.models.Comment):
        text = comment.body.replace("\n", " ")
        comments_list.append(text)
        karma_list.append(comment.score)
        type_list.append(comment_type)

        for reply in comment.replies:
            get_comments(reply, "Reply")

    elif isinstance(comment, praw.models.MoreComments):
        for more_comments in comment.comments():
            get_comments(more_comments, comment_type)

# Collect comments, karma scores, and types
for comment in post.comments:
    if len(comments_list) < 3000:
        get_comments(comment, "Comment")

In [62]:
# Create a dataframe from the collected data
data = {'Comment': comments_list, 'Karma': karma_list, 'Type': type_list}
df = pd.DataFrame(data)

In [63]:
df

,Comment,Karma,Type
0,just went through my phone and deleted all my ...,64,Comment
1,This shit is real.,1,Reply
2,SOMEONE FIND ME WHO WROTE THAT CHRISTMAS CAROL...,66,Comment
3,FOR REAL GREED WAS SO EASY,20,Reply
4,"YES THAT WAS AN ABSOLUTE GODSEND, EVEN THE EXT...",10,Reply
...,...,...,...
309,I'm afraid that I didn't get atleast a grade 7...,1,Comment
310,i swear the macbeth one was autogenerated or s...,1,Comment
311,I wonder how does it feel like doing this at t...,1,Comment
312,Lol I talked about how lady macbeth influenced...,1,Comment


In [46]:
#remove all emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

postemoji = []
for string in comments_list:
    postem = emoji_pattern.sub('', string)
    postemoji.append(postem)

In [47]:
postemoji

['just went through my phone and deleted all my tabs about macbeth it was the most satisfying thing ever honestly',
 'This shit is real.',
 'SOMEONE FIND ME WHO WROTE THAT CHRISTMAS CAROL QUESTION SO I CAN PLEASURE HIM',
 'FOR REAL GREED WAS SO EASY',
 'YES THAT WAS AN ABSOLUTE GODSEND, EVEN THE EXTRACT',
 'Yesssss',
 'IM NOT GAY BUT ILL JOIN YOU',
 'IT WAS SO GOOD I HAD SO MANY IDEAS',
 'I know it was so nice',
 'pleasure?',
 'if the j+h question isn’t on hyde i’m screwed completely',
 'you’re in luck then lol',
 'This aged well for you!',
 'honestly i’m so happy about it',
 'Same! Going into the exam I thought I would need to pull Lanyon or Utterson quotes out of my ass, but AQA doing god’s work with these questions!!',
 'Fr I was sweating over kingship and then it was just Macbeth LMAO',
 'I was still able to get lanyon, utterson, and Jekyll quotes for one of my points - the way all the other characters perceive him are the same and thus he is threatening yada yada',
 'Aged like win

In [48]:
#tokenize
posttoken = []
for i in postemoji:
    words = word_tokenize(i)
    posttoken.append(words)

In [49]:
posttoken

[['just',
  'went',
  'through',
  'my',
  'phone',
  'and',
  'deleted',
  'all',
  'my',
  'tabs',
  'about',
  'macbeth',
  'it',
  'was',
  'the',
  'most',
  'satisfying',
  'thing',
  'ever',
  'honestly'],
 ['This', 'shit', 'is', 'real', '.'],
 ['SOMEONE',
  'FIND',
  'ME',
  'WHO',
  'WROTE',
  'THAT',
  'CHRISTMAS',
  'CAROL',
  'QUESTION',
  'SO',
  'I',
  'CAN',
  'PLEASURE',
  'HIM'],
 ['FOR', 'REAL', 'GREED', 'WAS', 'SO', 'EASY'],
 ['YES',
  'THAT',
  'WAS',
  'AN',
  'ABSOLUTE',
  'GODSEND',
  ',',
  'EVEN',
  'THE',
  'EXTRACT'],
 ['Yesssss'],
 ['IM', 'NOT', 'GAY', 'BUT', 'ILL', 'JOIN', 'YOU'],
 ['IT', 'WAS', 'SO', 'GOOD', 'I', 'HAD', 'SO', 'MANY', 'IDEAS'],
 ['I', 'know', 'it', 'was', 'so', 'nice'],
 ['pleasure', '?'],
 ['if',
  'the',
  'j+h',
  'question',
  'isn',
  '’',
  't',
  'on',
  'hyde',
  'i',
  '’',
  'm',
  'screwed',
  'completely'],
 ['you', '’', 're', 'in', 'luck', 'then', 'lol'],
 ['This', 'aged', 'well', 'for', 'you', '!'],
 ['honestly', 'i', '’', 'm'

In [58]:
# remove all the punctuation
for i in posttoken:
    for index_j, j in enumerate(i):
        if j in punct:
            i.pop(index_j)

In [59]:
posttoken

[['just',
  'went',
  'through',
  'my',
  'phone',
  'and',
  'deleted',
  'all',
  'my',
  'tabs',
  'about',
  'macbeth',
  'it',
  'was',
  'the',
  'most',
  'satisfying',
  'thing',
  'ever',
  'honestly'],
 ['This', 'shit', 'is', 'real'],
 ['SOMEONE',
  'FIND',
  'ME',
  'WHO',
  'WROTE',
  'THAT',
  'CHRISTMAS',
  'CAROL',
  'QUESTION',
  'SO',
  'I',
  'CAN',
  'PLEASURE',
  'HIM'],
 ['FOR', 'REAL', 'GREED', 'WAS', 'SO', 'EASY'],
 ['YES', 'THAT', 'WAS', 'AN', 'ABSOLUTE', 'GODSEND', 'EVEN', 'THE', 'EXTRACT'],
 ['Yesssss'],
 ['IM', 'NOT', 'GAY', 'BUT', 'ILL', 'JOIN', 'YOU'],
 ['IT', 'WAS', 'SO', 'GOOD', 'I', 'HAD', 'SO', 'MANY', 'IDEAS'],
 ['I', 'know', 'it', 'was', 'so', 'nice'],
 ['pleasure'],
 ['if',
  'the',
  'j+h',
  'question',
  'isn',
  '’',
  't',
  'on',
  'hyde',
  'i',
  '’',
  'm',
  'screwed',
  'completely'],
 ['you', '’', 're', 'in', 'luck', 'then', 'lol'],
 ['This', 'aged', 'well', 'for', 'you'],
 ['honestly', 'i', '’', 'm', 'so', 'happy', 'about', 'it'],
 ['Sa

In [52]:
#lemmatize
postlemma = []

for i in posttoken: 
    postlemma.append([lemmatizer.lemmatize(j) for j in i])

In [53]:
postlemma

[['just',
  'went',
  'through',
  'my',
  'phone',
  'and',
  'deleted',
  'all',
  'my',
  'tab',
  'about',
  'macbeth',
  'it',
  'wa',
  'the',
  'most',
  'satisfying',
  'thing',
  'ever',
  'honestly'],
 ['This', 'shit', 'is', 'real'],
 ['SOMEONE',
  'FIND',
  'ME',
  'WHO',
  'WROTE',
  'THAT',
  'CHRISTMAS',
  'CAROL',
  'QUESTION',
  'SO',
  'I',
  'CAN',
  'PLEASURE',
  'HIM'],
 ['FOR', 'REAL', 'GREED', 'WAS', 'SO', 'EASY'],
 ['YES', 'THAT', 'WAS', 'AN', 'ABSOLUTE', 'GODSEND', 'EVEN', 'THE', 'EXTRACT'],
 ['Yesssss'],
 ['IM', 'NOT', 'GAY', 'BUT', 'ILL', 'JOIN', 'YOU'],
 ['IT', 'WAS', 'SO', 'GOOD', 'I', 'HAD', 'SO', 'MANY', 'IDEAS'],
 ['I', 'know', 'it', 'wa', 'so', 'nice'],
 ['pleasure'],
 ['if',
  'the',
  'j+h',
  'question',
  'isn',
  '’',
  't',
  'on',
  'hyde',
  'i',
  '’',
  'm',
  'screwed',
  'completely'],
 ['you', '’', 're', 'in', 'luck', 'then', 'lol'],
 ['This', 'aged', 'well', 'for', 'you'],
 ['honestly', 'i', '’', 'm', 'so', 'happy', 'about', 'it'],
 ['Same'

In [54]:
#lower all cases
postfin1 = [[j.lower() for j in i] for i in postlemma]

In [55]:
postfin1

[['just',
  'went',
  'through',
  'my',
  'phone',
  'and',
  'deleted',
  'all',
  'my',
  'tab',
  'about',
  'macbeth',
  'it',
  'wa',
  'the',
  'most',
  'satisfying',
  'thing',
  'ever',
  'honestly'],
 ['this', 'shit', 'is', 'real'],
 ['someone',
  'find',
  'me',
  'who',
  'wrote',
  'that',
  'christmas',
  'carol',
  'question',
  'so',
  'i',
  'can',
  'pleasure',
  'him'],
 ['for', 'real', 'greed', 'was', 'so', 'easy'],
 ['yes', 'that', 'was', 'an', 'absolute', 'godsend', 'even', 'the', 'extract'],
 ['yesssss'],
 ['im', 'not', 'gay', 'but', 'ill', 'join', 'you'],
 ['it', 'was', 'so', 'good', 'i', 'had', 'so', 'many', 'ideas'],
 ['i', 'know', 'it', 'wa', 'so', 'nice'],
 ['pleasure'],
 ['if',
  'the',
  'j+h',
  'question',
  'isn',
  '’',
  't',
  'on',
  'hyde',
  'i',
  '’',
  'm',
  'screwed',
  'completely'],
 ['you', '’', 're', 'in', 'luck', 'then', 'lol'],
 ['this', 'aged', 'well', 'for', 'you'],
 ['honestly', 'i', '’', 'm', 'so', 'happy', 'about', 'it'],
 ['same'

In [60]:
# replace the old comments with the new pre-processed comments
data = {'Comment': postfin1, 'Karma': karma_list, 'Type': type_list}
post1 = pd.DataFrame(data)

In [61]:
post1

,Comment,Karma,Type
0,"[just, went, through, my, phone, and, deleted,...",64,Comment
1,"[this, shit, is, real]",1,Reply
2,"[someone, find, me, who, wrote, that, christma...",66,Comment
3,"[for, real, greed, was, so, easy]",20,Reply
4,"[yes, that, was, an, absolute, godsend, even, ...",10,Reply
...,...,...,...
309,"[i, 'm, afraid, that, i, did, n't, get, atleas...",1,Comment
310,"[i, swear, the, macbeth, one, wa, autogenerate...",1,Comment
311,"[i, wonder, how, doe, it, feel, like, doing, t...",1,Comment
312,"[lol, i, talked, about, how, lady, macbeth, in...",1,Comment


#  1.2 Dataset 2

In [64]:
# Post ID
post_id = "13q6me2"
post = reddit.submission(id=post_id)

In [80]:
# Empty lists to store data
comments_list = []
karma_list = []
type_list = []

# Function to recursively collect comments, karma scores, and types
def get_comments(comment, comment_type):
    if isinstance(comment, praw.models.Comment):
        text = comment.body.replace("\n", " ")
        comments_list.append(text)
        karma_list.append(comment.score)
        type_list.append(comment_type)

        for reply in comment.replies:
            get_comments(reply, "Reply")

    elif isinstance(comment, praw.models.MoreComments):
        for more_comments in comment.comments():
            get_comments(more_comments, comment_type)

# Skip the first comment and collect comments, karma scores, and types
for comment in post.comments[1:]:
    if len(comments_list) < 3000:
        get_comments(comment, "Comment")

In [81]:
# Create a dataframe from the collected data
data = {'Comment': comments_list, 'Karma': karma_list, 'Type': type_list}
df = pd.DataFrame(data)

In [82]:
df

,Comment,Karma,Type
0,I saw too many memes about Edna being the ques...,151,Comment
1,smart asf,30,Reply
2,Huge brain,6,Reply
3,bro is the thinker,1,Reply
4,i mean it's alright,-2,Reply
...,...,...,...
498,I wrote about the dual nature of the wall wall...,1,Comment
499,I hope My last duchess comes up,-1,Comment
500,"OMG IT CAME UP%,,!!!1!1!!1",0,Reply
501,MY LAST DUCHESS such a crap poem,0,Comment


In [83]:
#remove all emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

postemoji = []
for string in comments_list:
    postem = emoji_pattern.sub('', string)
    postemoji.append(postem)

In [84]:
postemoji

["I saw too many memes about Edna being the question that I wrote half a page about her for my AIC question on women.  'Priestley cleverly doesn't develop Edna's character as it is reminiscent of the dismissive attitude which the Birlings and other upper class families would have towards their servants and often just women.'",
 'smart asf',
 'Huge brain',
 'bro is the thinker',
 "i mean it's alright",
 'Mr everything English said that it might be my last duchess ',
 'LMAO guess what',
 'I gave up',
 'he guessed the gender question for AIC too',
 'Wow',
 'Bruh moment',
 'I just gave up',
 'Nah because there is no way he can guess that',
 'bro fr that was one of the only poems i had no idea and it came up ffs',
 'DING DING DING',
 'SHIT SHIT SHIT',
 'That dude carries my lit grade',
 'Same',
 'Why not compare the abusive relationships between the duke and the duchees, with the abusive relationships between the soldiers and the government in Exposure or Bayonet Charge?',
 'I talked about 

In [85]:
#tokenize
posttoken = []
for i in postemoji:
    words = word_tokenize(i)
    posttoken.append(words)

In [86]:
posttoken

[['I',
  'saw',
  'too',
  'many',
  'memes',
  'about',
  'Edna',
  'being',
  'the',
  'question',
  'that',
  'I',
  'wrote',
  'half',
  'a',
  'page',
  'about',
  'her',
  'for',
  'my',
  'AIC',
  'question',
  'on',
  'women',
  '.',
  "'Priestley",
  'cleverly',
  'does',
  "n't",
  'develop',
  'Edna',
  "'s",
  'character',
  'as',
  'it',
  'is',
  'reminiscent',
  'of',
  'the',
  'dismissive',
  'attitude',
  'which',
  'the',
  'Birlings',
  'and',
  'other',
  'upper',
  'class',
  'families',
  'would',
  'have',
  'towards',
  'their',
  'servants',
  'and',
  'often',
  'just',
  'women',
  '.',
  "'"],
 ['smart', 'asf'],
 ['Huge', 'brain'],
 ['bro', 'is', 'the', 'thinker'],
 ['i', 'mean', 'it', "'s", 'alright'],
 ['Mr',
  'everything',
  'English',
  'said',
  'that',
  'it',
  'might',
  'be',
  'my',
  'last',
  'duchess'],
 ['LMAO', 'guess', 'what'],
 ['I', 'gave', 'up'],
 ['he', 'guessed', 'the', 'gender', 'question', 'for', 'AIC', 'too'],
 ['Wow'],
 ['Bruh', 'm

In [87]:
# remove all the punctuation
for i in posttoken:
    for index_j, j in enumerate(i):
        if j in punct:
            i.pop(index_j)

In [88]:
posttoken

[['I',
  'saw',
  'too',
  'many',
  'memes',
  'about',
  'Edna',
  'being',
  'the',
  'question',
  'that',
  'I',
  'wrote',
  'half',
  'a',
  'page',
  'about',
  'her',
  'for',
  'my',
  'AIC',
  'question',
  'on',
  'women',
  "'Priestley",
  'cleverly',
  'does',
  "n't",
  'develop',
  'Edna',
  "'s",
  'character',
  'as',
  'it',
  'is',
  'reminiscent',
  'of',
  'the',
  'dismissive',
  'attitude',
  'which',
  'the',
  'Birlings',
  'and',
  'other',
  'upper',
  'class',
  'families',
  'would',
  'have',
  'towards',
  'their',
  'servants',
  'and',
  'often',
  'just',
  'women',
  "'"],
 ['smart', 'asf'],
 ['Huge', 'brain'],
 ['bro', 'is', 'the', 'thinker'],
 ['i', 'mean', 'it', "'s", 'alright'],
 ['Mr',
  'everything',
  'English',
  'said',
  'that',
  'it',
  'might',
  'be',
  'my',
  'last',
  'duchess'],
 ['LMAO', 'guess', 'what'],
 ['I', 'gave', 'up'],
 ['he', 'guessed', 'the', 'gender', 'question', 'for', 'AIC', 'too'],
 ['Wow'],
 ['Bruh', 'moment'],
 ['I'

In [89]:
#lemmatize
postlemma = []

for i in posttoken: 
    postlemma.append([lemmatizer.lemmatize(j) for j in i])

In [90]:
postlemma

[['I',
  'saw',
  'too',
  'many',
  'meme',
  'about',
  'Edna',
  'being',
  'the',
  'question',
  'that',
  'I',
  'wrote',
  'half',
  'a',
  'page',
  'about',
  'her',
  'for',
  'my',
  'AIC',
  'question',
  'on',
  'woman',
  "'Priestley",
  'cleverly',
  'doe',
  "n't",
  'develop',
  'Edna',
  "'s",
  'character',
  'a',
  'it',
  'is',
  'reminiscent',
  'of',
  'the',
  'dismissive',
  'attitude',
  'which',
  'the',
  'Birlings',
  'and',
  'other',
  'upper',
  'class',
  'family',
  'would',
  'have',
  'towards',
  'their',
  'servant',
  'and',
  'often',
  'just',
  'woman',
  "'"],
 ['smart', 'asf'],
 ['Huge', 'brain'],
 ['bro', 'is', 'the', 'thinker'],
 ['i', 'mean', 'it', "'s", 'alright'],
 ['Mr',
  'everything',
  'English',
  'said',
  'that',
  'it',
  'might',
  'be',
  'my',
  'last',
  'duchess'],
 ['LMAO', 'guess', 'what'],
 ['I', 'gave', 'up'],
 ['he', 'guessed', 'the', 'gender', 'question', 'for', 'AIC', 'too'],
 ['Wow'],
 ['Bruh', 'moment'],
 ['I', 'jus

In [91]:
#lower all cases
postfin2 = [[j.lower() for j in i] for i in postlemma]

In [92]:
postfin2

[['i',
  'saw',
  'too',
  'many',
  'meme',
  'about',
  'edna',
  'being',
  'the',
  'question',
  'that',
  'i',
  'wrote',
  'half',
  'a',
  'page',
  'about',
  'her',
  'for',
  'my',
  'aic',
  'question',
  'on',
  'woman',
  "'priestley",
  'cleverly',
  'doe',
  "n't",
  'develop',
  'edna',
  "'s",
  'character',
  'a',
  'it',
  'is',
  'reminiscent',
  'of',
  'the',
  'dismissive',
  'attitude',
  'which',
  'the',
  'birlings',
  'and',
  'other',
  'upper',
  'class',
  'family',
  'would',
  'have',
  'towards',
  'their',
  'servant',
  'and',
  'often',
  'just',
  'woman',
  "'"],
 ['smart', 'asf'],
 ['huge', 'brain'],
 ['bro', 'is', 'the', 'thinker'],
 ['i', 'mean', 'it', "'s", 'alright'],
 ['mr',
  'everything',
  'english',
  'said',
  'that',
  'it',
  'might',
  'be',
  'my',
  'last',
  'duchess'],
 ['lmao', 'guess', 'what'],
 ['i', 'gave', 'up'],
 ['he', 'guessed', 'the', 'gender', 'question', 'for', 'aic', 'too'],
 ['wow'],
 ['bruh', 'moment'],
 ['i', 'jus

In [93]:
# replace the old comments with the new pre-processed comments
data = {'Comment': postfin2, 'Karma': karma_list, 'Type': type_list}
post2 = pd.DataFrame(data)

In [94]:
post2

,Comment,Karma,Type
0,"[i, saw, too, many, meme, about, edna, being, ...",151,Comment
1,"[smart, asf]",30,Reply
2,"[huge, brain]",6,Reply
3,"[bro, is, the, thinker]",1,Reply
4,"[i, mean, it, 's, alright]",-2,Reply
...,...,...,...
498,"[i, wrote, about, the, dual, nature, of, the, ...",1,Comment
499,"[i, hope, my, last, duchess, come, up]",-1,Comment
500,"[omg, it, came, up, ,, !, !, 1, 1, !, 1]",0,Reply
501,"[my, last, duchess, such, a, crap, poem]",0,Comment


#  1.3 Dataset 3

In [148]:
# Post ID
post_id = "4kn5fw"
post = reddit.submission(id=post_id)

In [149]:
# Empty lists to store data
comments_list = []
karma_list = []
type_list = []

# Function to recursively collect comments, karma scores, and types
def get_comments(comment, comment_type):
    if isinstance(comment, praw.models.Comment):
        text = comment.body.replace("\n", " ")
        comments_list.append(text)
        karma_list.append(comment.score)
        type_list.append(comment_type)

        for reply in comment.replies:
            get_comments(reply, "Reply")

    elif isinstance(comment, praw.models.MoreComments):
        for more_comments in comment.comments():
            get_comments(more_comments, comment_type)

# Skip the first comment and collect comments, karma scores, and types
for comment in post.comments[1:]:
    if len(comments_list) < 3000:
        get_comments(comment, "Comment")

In [185]:
# Create a dataframe from the collected data
data = {'Comment': comments_list, 'Karma': karma_list, 'Type': type_list}
df = pd.DataFrame(data)

In [186]:
df

,Comment,Karma,Type
0,[deleted],20,Comment
1,Were you the only one that felt that way or we...,2,Reply
2,I thought I messed up completely last year but...,1,Reply
3,FUCK MY LIFE QUESTIONS WERE EASY AF BUT MY ANS...,20,Comment
4,Fucking forgot to write about effect on reader...,11,Comment
...,...,...,...
91,Do you mean you put the wrong question number?...,1,Reply
92,Yeah. I wrote 08 but answered 07.,2,Reply
93,"Should be fine, so long as your answer made se...",3,Reply
94,[deleted],0,Comment


In [191]:
#Using drop() to delete rows based on column value
df.drop(df[df['Comment'] == '[deleted]'].index, inplace = True)

In [192]:
df

,Comment,Karma,Type
1,Were you the only one that felt that way or we...,2,Reply
2,I thought I messed up completely last year but...,1,Reply
3,FUCK MY LIFE QUESTIONS WERE EASY AF BUT MY ANS...,20,Comment
4,Fucking forgot to write about effect on reader...,11,Comment
6,English isn't marked in the same specific way ...,7,Reply
...,...,...,...
90,oh shite thats unlucky,3,Reply
91,Do you mean you put the wrong question number?...,1,Reply
92,Yeah. I wrote 08 but answered 07.,2,Reply
93,"Should be fine, so long as your answer made se...",3,Reply


In [211]:
karma = df['Karma'].tolist()
type = df['Type'].tolist()

In [193]:
comments = df['Comment'].tolist()

In [194]:
comments

["Were you the only one that felt that way or were there others?  I thought I did quite meh in the language exam but ended up with an A* but I don't know if that will happen for this exam too.",
 'I thought I messed up completely last year but I ended up getting an a*',
 'FUCK MY LIFE QUESTIONS WERE EASY AF BUT MY ANSWERS DESERVED TO DO AN EVA',
 'Fucking forgot to write about effect on reader kill me ',
 "English isn't marked in the same specific way other subjects are marked. You may lose next to nothing if you fluently wrote other parts of the essay. Don't worry bro, keep your head up and move on to the next exam",
 'Really? Oh thank fuck.',
 'just remember Point Example Explain Analyse Readers response ',
 'I did AIC and OMAM.  The questions where easy enough, but my execution for AIC was, in my opinion, pretty bad. I tried to follow  /u/jackcarr45 structure for AIC and messed up fairly hard.  OMAM went a lot smoother for me. But I think I spent too much time on the context of 1930

In [195]:
#remove all emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

postemoji = []
for string in comments:
    postem = emoji_pattern.sub('', string)
    postemoji.append(postem)

In [196]:
postemoji

["Were you the only one that felt that way or were there others?  I thought I did quite meh in the language exam but ended up with an A* but I don't know if that will happen for this exam too.",
 'I thought I messed up completely last year but I ended up getting an a*',
 'FUCK MY LIFE QUESTIONS WERE EASY AF BUT MY ANSWERS DESERVED TO DO AN EVA',
 'Fucking forgot to write about effect on reader kill me ',
 "English isn't marked in the same specific way other subjects are marked. You may lose next to nothing if you fluently wrote other parts of the essay. Don't worry bro, keep your head up and move on to the next exam",
 'Really? Oh thank fuck.',
 'just remember Point Example Explain Analyse Readers response ',
 'I did AIC and OMAM.  The questions where easy enough, but my execution for AIC was, in my opinion, pretty bad. I tried to follow  /u/jackcarr45 structure for AIC and messed up fairly hard.  OMAM went a lot smoother for me. But I think I spent too much time on the context of 1930

In [197]:
#tokenize
posttoken = []
for i in postemoji:
    words = word_tokenize(i)
    posttoken.append(words)

In [198]:
posttoken

[['Were',
  'you',
  'the',
  'only',
  'one',
  'that',
  'felt',
  'that',
  'way',
  'or',
  'were',
  'there',
  'others',
  '?',
  'I',
  'thought',
  'I',
  'did',
  'quite',
  'meh',
  'in',
  'the',
  'language',
  'exam',
  'but',
  'ended',
  'up',
  'with',
  'an',
  'A',
  '*',
  'but',
  'I',
  'do',
  "n't",
  'know',
  'if',
  'that',
  'will',
  'happen',
  'for',
  'this',
  'exam',
  'too',
  '.'],
 ['I',
  'thought',
  'I',
  'messed',
  'up',
  'completely',
  'last',
  'year',
  'but',
  'I',
  'ended',
  'up',
  'getting',
  'an',
  'a',
  '*'],
 ['FUCK',
  'MY',
  'LIFE',
  'QUESTIONS',
  'WERE',
  'EASY',
  'AF',
  'BUT',
  'MY',
  'ANSWERS',
  'DESERVED',
  'TO',
  'DO',
  'AN',
  'EVA'],
 ['Fucking',
  'forgot',
  'to',
  'write',
  'about',
  'effect',
  'on',
  'reader',
  'kill',
  'me'],
 ['English',
  'is',
  "n't",
  'marked',
  'in',
  'the',
  'same',
  'specific',
  'way',
  'other',
  'subjects',
  'are',
  'marked',
  '.',
  'You',
  'may',
  'lose'

In [199]:
# remove all the punctuation
for i in posttoken:
    for index_j, j in enumerate(i):
        if j in punct:
            i.pop(index_j)

In [200]:
posttoken

[['Were',
  'you',
  'the',
  'only',
  'one',
  'that',
  'felt',
  'that',
  'way',
  'or',
  'were',
  'there',
  'others',
  'I',
  'thought',
  'I',
  'did',
  'quite',
  'meh',
  'in',
  'the',
  'language',
  'exam',
  'but',
  'ended',
  'up',
  'with',
  'an',
  'A',
  'but',
  'I',
  'do',
  "n't",
  'know',
  'if',
  'that',
  'will',
  'happen',
  'for',
  'this',
  'exam',
  'too'],
 ['I',
  'thought',
  'I',
  'messed',
  'up',
  'completely',
  'last',
  'year',
  'but',
  'I',
  'ended',
  'up',
  'getting',
  'an',
  'a'],
 ['FUCK',
  'MY',
  'LIFE',
  'QUESTIONS',
  'WERE',
  'EASY',
  'AF',
  'BUT',
  'MY',
  'ANSWERS',
  'DESERVED',
  'TO',
  'DO',
  'AN',
  'EVA'],
 ['Fucking',
  'forgot',
  'to',
  'write',
  'about',
  'effect',
  'on',
  'reader',
  'kill',
  'me'],
 ['English',
  'is',
  "n't",
  'marked',
  'in',
  'the',
  'same',
  'specific',
  'way',
  'other',
  'subjects',
  'are',
  'marked',
  'You',
  'may',
  'lose',
  'next',
  'to',
  'nothing',
  

In [201]:
#lemmatize
postlemma = []

for i in posttoken: 
    postlemma.append([lemmatizer.lemmatize(j) for j in i])

In [202]:
postlemma

[['Were',
  'you',
  'the',
  'only',
  'one',
  'that',
  'felt',
  'that',
  'way',
  'or',
  'were',
  'there',
  'others',
  'I',
  'thought',
  'I',
  'did',
  'quite',
  'meh',
  'in',
  'the',
  'language',
  'exam',
  'but',
  'ended',
  'up',
  'with',
  'an',
  'A',
  'but',
  'I',
  'do',
  "n't",
  'know',
  'if',
  'that',
  'will',
  'happen',
  'for',
  'this',
  'exam',
  'too'],
 ['I',
  'thought',
  'I',
  'messed',
  'up',
  'completely',
  'last',
  'year',
  'but',
  'I',
  'ended',
  'up',
  'getting',
  'an',
  'a'],
 ['FUCK',
  'MY',
  'LIFE',
  'QUESTIONS',
  'WERE',
  'EASY',
  'AF',
  'BUT',
  'MY',
  'ANSWERS',
  'DESERVED',
  'TO',
  'DO',
  'AN',
  'EVA'],
 ['Fucking',
  'forgot',
  'to',
  'write',
  'about',
  'effect',
  'on',
  'reader',
  'kill',
  'me'],
 ['English',
  'is',
  "n't",
  'marked',
  'in',
  'the',
  'same',
  'specific',
  'way',
  'other',
  'subject',
  'are',
  'marked',
  'You',
  'may',
  'lose',
  'next',
  'to',
  'nothing',
  '

In [203]:
#lower all cases
postfin3 = [[j.lower() for j in i] for i in postlemma]

In [204]:
postfin3

[['were',
  'you',
  'the',
  'only',
  'one',
  'that',
  'felt',
  'that',
  'way',
  'or',
  'were',
  'there',
  'others',
  'i',
  'thought',
  'i',
  'did',
  'quite',
  'meh',
  'in',
  'the',
  'language',
  'exam',
  'but',
  'ended',
  'up',
  'with',
  'an',
  'a',
  'but',
  'i',
  'do',
  "n't",
  'know',
  'if',
  'that',
  'will',
  'happen',
  'for',
  'this',
  'exam',
  'too'],
 ['i',
  'thought',
  'i',
  'messed',
  'up',
  'completely',
  'last',
  'year',
  'but',
  'i',
  'ended',
  'up',
  'getting',
  'an',
  'a'],
 ['fuck',
  'my',
  'life',
  'questions',
  'were',
  'easy',
  'af',
  'but',
  'my',
  'answers',
  'deserved',
  'to',
  'do',
  'an',
  'eva'],
 ['fucking',
  'forgot',
  'to',
  'write',
  'about',
  'effect',
  'on',
  'reader',
  'kill',
  'me'],
 ['english',
  'is',
  "n't",
  'marked',
  'in',
  'the',
  'same',
  'specific',
  'way',
  'other',
  'subject',
  'are',
  'marked',
  'you',
  'may',
  'lose',
  'next',
  'to',
  'nothing',
  '

In [212]:
# # replace the old comments with the new pre-processed comments
data = {'Comment': postfin3, 'Karma': karma, 'Type': type}
post3 = pd.DataFrame(data)

In [214]:
post3

,Comment,Karma,Type
0,"[were, you, the, only, one, that, felt, that, ...",2,Reply
1,"[i, thought, i, messed, up, completely, last, ...",1,Reply
2,"[fuck, my, life, questions, were, easy, af, bu...",20,Comment
3,"[fucking, forgot, to, write, about, effect, on...",11,Comment
4,"[english, is, n't, marked, in, the, same, spec...",7,Reply
...,...,...,...
86,"[oh, shite, thats, unlucky]",3,Reply
87,"[do, you, mean, you, put, the, wrong, question...",1,Reply
88,"[yeah, i, wrote, 08, but, answered, 07]",2,Reply
89,"[should, be, fine, so, long, a, your, answer, ...",3,Reply


#  2 - Combine to create one dataset

In [179]:
# the three dataframes should be combined to create one 'reddit' dataset

In [215]:
reddit = pd.concat([post1, post2, post3], axis=0)

In [216]:
reddit

,Comment,Karma,Type
0,"[just, went, through, my, phone, and, deleted,...",64,Comment
1,"[this, shit, is, real]",1,Reply
2,"[someone, find, me, who, wrote, that, christma...",66,Comment
3,"[for, real, greed, was, so, easy]",20,Reply
4,"[yes, that, was, an, absolute, godsend, even, ...",10,Reply
...,...,...,...
86,"[oh, shite, thats, unlucky]",3,Reply
87,"[do, you, mean, you, put, the, wrong, question...",1,Reply
88,"[yeah, i, wrote, 08, but, answered, 07]",2,Reply
89,"[should, be, fine, so, long, a, your, answer, ...",3,Reply


In [217]:
len(reddit)

908

#  3 - Anonymise dataset

In [45]:
# as these are comments from individual users, these must be anonymised.
# This is to protect their identity and comply with Ethics Procedures.

In [220]:
# define a function to scramble the word order in a string
def scramble_words(text):
    words = ' '.join(text).split()
    random.shuffle(words)
    return ' '.join(words)

# apply this to the reddit dataframe column 'Comment'
reddit['Comment'] = reddit['Comment'].apply(scramble_words)

In [221]:
reddit

,Comment,Karma,Type
0,phone through about tab it most ever my honest...,64,Comment
1,real shit this is,1,Reply
2,carol that who i him christmas someone so can ...,66,Comment
3,real was so easy for greed,20,Reply
4,godsend an even extract yes the that was absolute,10,Reply
...,...,...,...
86,unlucky shite thats oh,3,Reply
87,put if question wrong answering you examiner c...,1,Reply
88,07 answered 08 i but yeah wrote,2,Reply
89,fine long your 're answer talk should to your ...,3,Reply


#  4 - Save Finalised Dataset

In [222]:
# save the dataset to be able to use in analysis

In [223]:
#export to excel file
reddit.to_excel('1reddit.xlsx')

In [224]:
#export to csv file
reddit.to_csv('1reddit.csv', index=False)